In [ ]:
#runs the windown analysis basic. look for 'running_window_analysis.ipynb' for better version

In [ ]:
#imports
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import seaborn as sns
#from dataClasses import Trial,Neuron
from sklearn.feature_selection import RFECV
from sklearn import svm
from sklearn.model_selection import cross_validate, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report
#import intertools 
# Import Libraries
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from imblearn.pipeline import  Pipeline

from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import boto3
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy import stats
import numpy as np
from sklearn.metrics import confusion_matrix
from imblearn.pipeline import Pipeline

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline

from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.pipeline import Pipeline

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        
        # Defining the RNN layer
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        
        # RNN forward pass
        out, _ = self.rnn(x)
        
        # Use only the last time step's output
        out = out[:, -1]  # Take the output from the last sequence step
        
        # Fully connected layer
        out = self.fc(out)
        
        return out


In [ ]:
def get_data_ready(brain_area, type_of_analysis):
    all_trials=np.load("processed_data/"+ brain_area + "_all_trials.npy",allow_pickle=True)
    trial_list=np.empty((all_trials.shape[0],3))
    trial_dff=np.empty((all_trials.shape[0],41)) 
    trial_licks=np.empty((all_trials.shape[0],41))
    # Here we get the data based on the conditions that we want (i.e: if it is is lick, or go) 
    
    for trial in range(all_trials.shape[0]):
        
        trial_list[trial,0]=all_trials[trial].neuron_num
    
        
        if type_of_analysis=="GO":
            # for labelling go nogo
            if (all_trials[trial].trial_type)=="go":
                trial_list[trial,1]=1
            elif (all_trials[trial].trial_type)=="nogo":
                trial_list[trial,1]=0
            else:
                print("gone wrong on neuron ",all_trials[trial].neuron_num )
        elif type_of_analysis=="LICK":    
            #for labelling lick nolick
            if (all_trials[trial].trial_outcome)=="FA":
                trial_list[trial,1]=1
            elif (all_trials[trial].trial_outcome)=="Hit":
                trial_list[trial,1]=1
            elif (all_trials[trial].trial_outcome)=="Miss":
                trial_list[trial,1]=0
            elif (all_trials[trial].trial_outcome)=="CR":
                trial_list[trial,1]=0
            else:
                print("gone wrong on neuron ",all_trials[trial].neuron_num )
        else:
            print("Type of analysis can only be 'GO' or 'LICK'")
        
        trial_list[trial,2]=all_trials[trial].mouse_id
        if (all_trials[trial].trial_outcome)=="FA" or (all_trials[trial].trial_outcome)=="Hit":
            trial_dff[trial,0:41]=all_trials[trial].dff 
            #rebase  dff 
            trial_licks[trial,:]=all_trials[trial].licks
            lick_start=np.argmax(trial_licks[trial,:]==1,axis=0)
            trial_dff_rel_licks=np.zeros(trial_licks.shape[1])
            new_start=lick_start-4
            if new_start>=0:                    
                # move the array of licks left so that timeseries starst at lick time
                trial_dff_rel_licks[0:trial_licks.shape[1]-new_start]=all_trials[trial].dff[new_start: ]
                ## need to mske the last few timesteps on the new series = final timestep on original series
                trial_dff_rel_licks[trial_licks.shape[1]-new_start :]=all_trials[trial].dff[-1]
            else: 
                # move the array of licks right so that timeseries starst at lick time
                trial_dff_rel_licks[0-new_start :]=all_trials[trial].dff[0:all_trials[trial].dff.shape[0]+new_start]
                ## need to mske the first few timesteps on the new series = first timestep on original series
                trial_dff_rel_licks[0:-new_start]=all_trials[trial].dff[ 0 ]
                #trial_dff[trial,41:82]=trial_dff_rel_licks
        else:
            trial_dff[trial,0:41]=all_trials[trial].dff 
            #trial_dff[trial,41:82]=all_trials[trial].dff
    return trial_list, trial_dff


In [ ]:
# run the analysis for different windows for each neuron 
num_permutations = 1000
results_window=[]
# Brain areas and analysis types
brain_area = ("S")
type_of_analysis = "GO"

for brain in brain_area:
    trial_list, trial_dff = get_data_ready("S1naive", type_of_analysis)
    neuron_list = np.unique(trial_list[:, 0]).reshape(-1, 1)
    
    print("Num of neurons before filter=", neuron_list.shape[0])
    
    #window_of_analysis=["REWARD", "STIM"]
    size_of_window=[1]
    
    for size in size_of_window:
        window_start=np.arange(0,41,size)
        for start in window_start: 
    
            for neuron in neuron_list:
                print("\n", neuron, "XXXXXXXXXXXXXXXXXXXX")
                    
                    # Extract neuron-specific data
                neuron_data_list = trial_list[trial_list[:, 0] == neuron].copy()
                neuron_data_dff = trial_dff[trial_list[:, 0] == neuron].copy()

                # get window specific data
                neuron_data_dff = neuron_data_dff[:,start:start+size] #smaller (all the way to 40)
            
                X = neuron_data_dff.copy()
                y = neuron_data_list[:, 1]
        
               
                # Train-test split
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
                # Apply PCA to reduce to 10 components
                n_components=min(size,5)
                pcaTrain = PCA(n_components=n_components)
                pcaTest=PCA(n_components=n_components)
                X_train_pca = pcaTrain.fit_transform(X_train)
                X_test_pca = pcaTest.fit_transform(X_test)
                
                # Reshape PCA output to 3D for RNN input
                # New shape: (num_samples,num_components,1)
                num_components = X_train_pca.shape[1]
                X_train_pca = X_train_pca.reshape(X_train_pca.shape[0], num_components, 1)
                X_test_pca = X_test_pca.reshape(X_test_pca.shape[0], num_components, 1)
    
    
                # Convert to PyTorch tensors
                Xtrain = torch.from_numpy(X_train_pca).float()
                ytrain = torch.from_numpy(y_train).long()
                Xtest = torch.from_numpy(X_test_pca).float()
                ytest = torch.from_numpy(y_test).long()
        
                # Batch size, epochs, and iterations
                batch_size = 16
                n_iters = 1000
                num_epochs = int(n_iters // (len(X_train) // batch_size))
        
                train = TensorDataset(Xtrain, ytrain)
                test = TensorDataset(Xtest, ytest)
        
                train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, drop_last=True)
                test_loader = DataLoader(test, batch_size=batch_size, shuffle=True, drop_last=True)
        
                # Create the RNN model
                input_dim = X_train_pca.shape[-1]
                hidden_dim = 4
                layer_dim = 1
                output_dim = 1
        
                model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
        
                # Loss function and optimizer
                error = nn.BCEWithLogitsLoss()  # No class weights, since oversampling handles imbalance
                learning_rate = 0.01
                optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
                scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
        
                # Training loop for the original model
                train_accuracy_list = []
                test_accuracy_list = []
                test_loss_list = []
        
                for epoch in range(num_epochs):
                    model.train()
                    total_train, correct_train = 0, 0
        
                    for X_batch, y_batch in train_loader:
                        optimizer.zero_grad()
                        outputs = model(X_batch).squeeze()
                        loss = error(outputs, y_batch.float())
                        loss.backward()
                        optimizer.step()
        
                        probabilities = torch.sigmoid(outputs)
                        predicted = (probabilities > 0.5).long()
                        correct_train += (predicted.squeeze() == y_batch).sum().item()
                        total_train += y_batch.size(0)
        
                    train_accuracy = 100 * correct_train / total_train
                    train_accuracy_list.append(train_accuracy)
                    scheduler.step()
        
                    # Evaluate on test set
                    model.eval()
                    total_test, correct_test = 0, 0
                    with torch.no_grad():
                        for X_test_batch, y_test_batch in test_loader:
                            outputs = model(X_test_batch).squeeze()
                            probabilities = torch.sigmoid(outputs)
                            predicted = (probabilities > 0.5).long()
                            correct_test += (predicted == y_test_batch).sum().item()
                            total_test += y_test_batch.size(0)
                            loss_test = error(outputs, y_test_batch.float())
                            test_loss_list.append(loss_test)
        
                    test_accuracy = 100 * correct_test / float(total_test)
                    test_accuracy_list.append(test_accuracy)
        
                # Save the original model's test accuracy
                original_test_accuracy = test_accuracy
        
                # Random classifier: Match class distribution in the test set
                class_distribution = np.mean(y_test)  # proportion of class 1
        
                random_accuracies = []
                for _ in range(num_permutations):
                    y_random = np.random.choice([0, 1], size=len(y_test), p=[1 - class_distribution, class_distribution])
                    random_accuracy = accuracy_score(y_test, y_random)
                    random_accuracies.append(random_accuracy)
        
                # Calculate p-value using a t-test (testing original model vs. random classifiers)
                random_accuracies = np.array(random_accuracies)
                p_value = np.mean(random_accuracies >= original_test_accuracy/100)
        
                # Store results
                results_window.append({
                    'Brain Area': brain,
                    'Window Size': size,
                    'Window Start':start,
                    'Neuron': neuron,
                    'Train Accuracy': train_accuracy_list,
                    'Test Accuracy': original_test_accuracy,
                    'p value': p_value
                })
        
                print(f"Neuron {neuron}: Test Accuracy = {original_test_accuracy:.2f}%, p-value = {p_value:.4f}")
        
                # Visualize random classifier accuracy distribution
              #  plt.hist(random_accuracies, bins=20, color='blue', edgecolor='black')
               # plt.axvline(original_test_accuracy/100, color='red', linestyle='dashed', linewidth=2)
               # plt.title(f'Random Classifier Accuracy Distribution for Neuron {neuron}')
               # plt.xlabel('Accuracy')
               # plt.ylabel('Frequency')
               # plt.show()
        
        
             # Plot training vs test accuracy to examine potential overfitting or bias
               # plt.plot(train_accuracy_list, label='Training Accuracy')
                #plt.plot(test_accuracy_list, label='Test Accuracy')
                #plt.xlabel('Epochs')
                #plt.ylabel('Accuracy (%)')
                #plt.title('Training vs Test Accuracy')
                #plt.legend()
                #plt.show()
                
                # Check class-specific accuracy on the test set
                
                # Predict on the test set and calculate confusion matrix
                model.eval()
                with torch.no_grad():
                    outputs = model(Xtest).squeeze()
                    probabilities = torch.sigmoid(outputs)
                    predictions = (probabilities > 0.5).long()
                    
                conf_matrix = confusion_matrix(ytest, predictions)
                print("Confusion Matrix:\n", conf_matrix)
                
                # Calculate class-specific accuracy
                class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
                print("Class-specific Accuracy:", class_accuracy)

                #save 
                results_df = pd.DataFrame(results_window)

                # Save as CSV or Pickle
                results_df.to_csv('results_window_s1naive.csv', index=False)


In [ ]:
# Convert results_window to a DataFrame
results_df = pd.DataFrame(results_window)

# Display the first few rows
print(results_df.head())

# Check for missing or unexpected values   
print(results_df.info())
print(results_df.describe())



# Check unique values for grouping keys
print("Unique brain areas:", results_df['Brain Area'].unique())
print("Unique window sizes:", results_df['Window Size'].unique())
print("Unique window starts:", results_df['Window Start'].unique())